In [67]:
from dataclasses import dataclass
from typing import List, Tuple

import random
import vss
from crypto import G1, random_scalar, add, multiply, sum_points, CURVE_ORDER

In [76]:
def eval_public_poly(commitments, x):
    return sum_points([multiply(com, pow(x, j, CURVE_ORDER)) for j, com in enumerate(commitments)])

In [77]:
N = 5
T = 3

@dataclass
class Participant:
    idx: int
    secret: int
    shares: List[int]
    commitments: List[Tuple[int, int]]
    indexed_shares: List[Tuple[int, int]]
    secret_key: int = None
    public_key: Tuple[int, int] = None

In [101]:
P = []
for i in range(N):
    secret = random_scalar()
    shares, commitments = vss.share(secret, n, t)
    indexed_shares = list(zip(range(1, n + 1), shares))
    P.append(Participant(i + 1, secret, shares, commitments, indexed_shares))
    
for i in range(N):
    P[i].secret_key = sum(P[j].shares[i] for j in range(N)) % CURVE_ORDER
    P[i].public_key = multiply(G1, P[i].secret_key)
    assert P[i].public_key == sum_points([
        eval_public_poly(P[j].commitments, P[i].idx)
        for j in range(N)
    ])
    
    
secret_key = sum(P[i].secret for i in range(N)) % CURVE_ORDER
public_key = multiply(G1, secret_key)
assert public_key == sum_points([P[i].commitments[0] for i in range(N)])

G = random.sample(P, T)

assert P[i].secret == vss.recover(random.sample(P[i].indexed_shares, T))
assert secret_key == vss.recover([(G[k].idx, G[k].secret_key) for k in range(T)])

In [102]:
b = [random_scalar() for i in range(N)]
B = [multiply(G1, b[i]) for i in range(N)]

secret_key = ((secret_key) + sum(b)) % CURVE_ORDER
public_key = add(public_key, sum_points(B))

assert public_key == multiply(G1, secret_key)

for i in range(N):
    P[i].secret_key = (P[i].secret_key + sum(b)) % CURVE_ORDER
    P[i].public_key = add(P[i].public_key, sum_points(B))
    assert P[i].public_key == multiply(G1, P[i].secret_key)
            
assert secret_key == vss.recover([(G[k].idx, G[k].secret_key) for k in range(T)])